<a href="https://colab.research.google.com/github/soul11zz/imagecaption/blob/main/fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Train with Pytorch Lighning

In [1]:
import os
import os.path as osp
if osp.exists("imagecaption"):
    os.chdir("imagecaption")

In [2]:
!pip install -r train/requirements.txt

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-50if62m5
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-50if62m5
  Resolved https://github.com/huggingface/transformers.git to commit 5b49376202863d3798d2ff8a8ba61590542a1141
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 708.1/708.1 kB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 kB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 106.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.7/140.7 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 11.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.0/330.0 kB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os.path as osp
import sys
train_dir = osp.abspath("train")
if train_dir not in sys.path:
    sys.path.insert(0, train_dir)
    
import pytorch_lightning as pl
from huggingface_hub import HfApi, notebook_login


pl.seed_everything(42, workers=True)


Global seed set to 42


42

### Login to HuggingFace

In [4]:
notebook_login()

### Run PL Trainer


In [3]:
from argparse import Namespace
args = {
  "model": "microsoft/git-large",
  "dataset": "soul11zz/image-caption-site-description",
  "batch_size": 8,
  "epochs": 10,
  "metric": "meteor",
  "model_dir": "tmp/model",
  "tune_lr": False,
  "save_best": True,
  "test_best": True,
  "lr": 0.000630957344480193,
  "best_model": "soul11zz/image-caption-description-large"
}

args = Namespace(**args)


### Run Training

In [ ]:
from train import training_loop, save_best_model
best_model_path = training_loop(args)

Global seed set to 42
Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
2023-02-07-05-35-35  INFO       Tuning learning rate...
2023-02-07-05-35-35  INFO       Training...
2023-02-07-05-35-35  WARNING    Using custom data configuration soul11zz--image-caption-site-description-8760c1ca4bb9b173
2023-02-07-05-35-35  WARNING    Found cached dataset parquet (/root/.cache/huggingface/datasets/soul11zz___parquet/soul11zz--image-caption-site-description-8760c1ca4bb9b173/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
2023-02-07-05-35-35  WARNING    Using custom data configuration soul11zz--image-caption-site-description-8760c1ca4bb9b173
2023-02-07-05-35-35  WARNING    Found cached dataset parquet (/root/.cache/huggingface/datasets/soul11zz___parquet/soul11zz--image-caption-site-description-8760c1ca4bb9b173/0.0.0/2a3b91f

Training: 0it [00:00, ?it/s]

### Upload Best Model

In [ ]:
model_repo = "soul11zz/sample-best-model"
pl_best_model = save_best_model(model_repo, best_model_path)

### Evaluate Best Model

In [8]:
from eval import score_model
score_model(args.model, args.dataset, args.metric)